In [5]:
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import utils
import csv
from tqdm import tqdm
import multiprocessing
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
import pandas as pd

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/i574473/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
def remove_stopwords(text):
    stop_words = set(stopwords.words('portuguese'))
    word_tokens = word_tokenize(text)
    filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
    filtered_sentence = []
  
    for w in word_tokens:
        if w not in stop_words:
            filtered_sentence.append(w)  
    separator = ' '    
    return separator.join(filtered_sentence)

In [12]:
def tokenize_text(text):
    tokens = []
    text_filtered = remove_stopwords(text)
    for sent in nltk.sent_tokenize(text_filtered):
        for word in nltk.word_tokenize(sent):
            if len(word) < 2:
                continue
            tokens.append(word.lower())
    return tokens

In [8]:
df = pd.read_csv('tweets.csv')
df.head()

,Tweet.Text,User.Trusted
0,RT @jornalnacional: Operação Yanomami já destr...,True
1,Indivíduo escolhido para a função representari...,True
2,RT @jornalnacional: O #JN está no ar. 📺#BoaNoi...,True
3,Leia detalhes https://t.co/EnrFcSuZh9 #g1,True
4,A iniciativa custará R$ 70 milhões ao ano e se...,True


In [10]:
text_dataframe = df[['Tweet.Text', 'User.Trusted']]
text_dataframe.tail()

,Tweet.Text,User.Trusted
585,https://t.co/e6XUP55n5h,False
586,Brasileiro quer que Bolsonaro homenageie Neyma...,False
587,Gol de Richarlison pode ser o início da pacifi...,False
588,"Neymar se machuca, chora e torcida grita: 'E d...",False
589,Alexandre de Moraes resgata o número 22 e devo...,False


In [13]:
train_documents = []
#test_documents = []

for index, row in text_dataframe.iterrows():
    train_documents.append(TaggedDocument(words=tokenize_text(row['Tweet.Text']), tags=[row['User.Trusted']] ))
 
print(train_documents[0])

TaggedDocument<['rt', 'jornalnacional', 'operação', 'yanomami', 'destruiu', '200', 'acampamentos', 'garimpeiros', 'imagens', 'obtidas', 'exclusividade', 'jornalna…'], [True]>


In [14]:
cores = multiprocessing.cpu_count()

model_dbow = Doc2Vec(dm=0, vector_size=300, negative=5, hs=0, min_count=2, sample = 0, workers=cores, alpha=0.025, min_alpha=0.001)
model_dbow.build_vocab([x for x in tqdm(train_documents)])
train_documents  = utils.shuffle(train_documents)
model_dbow.train(train_documents,total_examples=len(train_documents), epochs=30)
def vector_for_learning(model, input_docs):
    sents = input_docs
#    targets, feature_vectors = zip(*[(doc.tags[0], model.infer_vector(doc.words, steps=20)) for doc in sents])
    targets, feature_vectors = zip(*[(doc.tags[0], model.infer_vector(doc.words)) for doc in sents])
    return targets, feature_vectors
model_dbow.save('./tweets.d2v')

100%|█████████████████████████████████████| 590/590 [00:00<00:00, 934884.53it/s]


In [15]:
from sklearn import preprocessing

y_train, X_train = vector_for_learning(model_dbow, train_documents)
# y_test, X_test = vector_for_learning(model_dbow, test_documents)

scaler = preprocessing.StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
# X_test_scaled = scaler.transform(X_test)


In [16]:
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler


model = LogisticRegression(n_jobs=1, C=1e5, max_iter=2000)
model.fit(X_train_scaled, y_train)

model.

LogisticRegression(C=100000.0, max_iter=2000, n_jobs=1)